In [1]:
import pandas as pd 
import torch
import numpy as np
from sklearn.preprocessing import MinMaxScaler

dfP = pd.read_csv("co2_327_45.csv")

In [2]:
### "Data preprocessing
df = np.array(dfP)
### "Data preprocessing
scaler = MinMaxScaler(feature_range=(0, 1))
df = scaler.fit_transform(df)

In [3]:
r1 = 2
r2 = 4
r3 = 5
p = 20
N = 45
Smp_size = 100
Pred_size = len(df)-Smp_size-p-100

In [4]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import math
import numpy as np
from scipy.linalg import svd
from torch.utils.data import Dataset, DataLoader
from numpy import linalg as LA
from time import perf_counter 
from torch.autograd import Variable
import torch.optim as optim

### '' Define the Network Structure: Linear
### Type 1. Linear Network
class NetLinear(nn.Module):

    def __init__(self, r1, r2, r3, p, N):
        
        self.r1 = r1
        self.r2 = r2
        self.r3 = r3
        self.p = p
        self.N = N
        super(NetLinear, self).__init__()
        # .conv1: 1 input matrix channel (N*P), r2 output channels, Nx1 convolution kernel
        # .conv2: 1 input matrix channel (1*P), r3 output channels, 1xr3 convolution kernel (kernel sharing)
        self.conv1 = nn.Conv2d(1, r2, kernel_size=(N, 1), bias=False) # stride is set to be (0,1) -> only move to the right
        self.conv2 = nn.Conv2d(1, r3, kernel_size=(1, p), bias=False)   # stride is set to be 0 -> no moving needed
        # an affine operation: y = Wx + b
        self.fc1 = nn.Linear(in_features=r2*r3, out_features=r1, bias=False)  # 6*6 from image dimension
        self.fc2 = nn.Linear(in_features=r1, out_features=N, bias=False)

    def forward(self, x):
        # Max pooling over a (2, 2) window
        x = self.conv1(x)
        z = self.conv2(x[:, :1, :, :])
        for i in range(1, x.shape[1]):
            z = torch.cat([z, self.conv2(x[:, i:(i+1), :, :])], dim = 1) #Flattening is achieved     
        z = z.view(-1, self.r2*self.r3) #-1 helps us figure out the batchsize
        x = self.fc1(z) #activation can be added on the inside as well
        x = self.fc2(x)
        return x
    
### Type 2. Nonlinear Network
class NetRelu(nn.Module):

    def __init__(self, r1, r2, r3, p, N):
        
        self.r1 = r1
        self.r2 = r2
        self.r3 = r3
        self.p = p
        self.N = N
        super(NetRelu, self).__init__()
        # .conv1: 1 input matrix channel (N*P), r2 output channels, Nx1 convolution kernel
        # .conv2: 1 input matrix channel (1*P), r3 output channels, 1xr3 convolution kernel (kernel sharing)
        self.conv1 = nn.Conv2d(1, r2, kernel_size=(N, 1), bias=True) # stride is set to be (0,1) -> only move to the right
        self.conv2 = nn.Conv2d(1, r3, kernel_size=(1, p), bias=True)   # stride is set to be 0 -> no moving needed
        # an affine operation: y = Wx + b
        self.fc1 = nn.Linear(in_features=r2*r3, out_features=r1, bias=True)  # 6*6 from image dimension
        self.fc2 = nn.Linear(in_features=r1, out_features=N, bias=True)

    def forward(self, x):
        # Max pooling over a (2, 2) window
        x = F.relu(self.conv1(x))
        z = F.relu(self.conv2(x[:, :1, :, :]))
        for i in range(1, x.shape[1]):
            z = torch.cat([z, self.conv2(x[:, i:(i+1), :, :])], dim = 1) #Flattening is achieved     
        z = z.view(-1, self.r2*self.r3) #-1 helps us figure out the batchsize
        x = self.fc1(z) #activation can be added on the inside as well
        x = self.fc2(x)
        return x
    
### Type 3. Nonlinear DW Network    
class NetDW(nn.Module):

    def __init__(self, r1, r2, r3, p, N):
        
        self.r1 = r1
        self.r2 = r2
        self.r3 = r3
        self.p = p
        self.N = N
        super(NetDW, self).__init__()
        # .conv1: 1 input matrix channel (N*P), r2 output channels, Nx1 convolution kernel
        # .conv2: 1 input matrix channel (1*P), r3 output channels, 1xr3 convolution kernel (kernel sharing)
        self.conv1 = nn.Conv2d(1, r2, kernel_size=(N, 1), bias=True) # stride is set to be (0,1) -> only move to the right
        self.conv2 = nn.Conv2d(1, r3, kernel_size=(1, p), bias=True)   # stride is set to be 0 -> no moving needed
        # an affine operation: y = Wx + b
        self.fc1 = nn.Linear(in_features=r2*r3, out_features=r1, bias=True)  # 6*6 from image dimension
        self.fc2 = nn.Linear(in_features=r1, out_features=N, bias=True)

    def forward(self, x):
        y1 = F.relu(self.conv1(x))
        y2 = F.relu(self.conv2(x))
        # first N, then p
        z1 = self.conv2(y1[:, :1, :, :])
        for i in range(1, y1.shape[1]):
            z1 = torch.cat([z1, self.conv2(y1[:, i:(i+1), :, :])], dim = 1) #Flattening is achieved     
        z1 = F.relu(z1.view(-1, self.r2*self.r3)) #-1 helps us figure out the batchsize
        # first p, then N
        z2 = F.relu(self.conv1(y2[:, :1, :, :]))
        for i in range(1, y2.shape[1]):
            z2 = torch.cat([z2, self.conv1(y2[:, i:(i+1), :, :])], dim = 2) #Flattening is achieved     
        z2 = F.relu(z2.view(-1, self.r2*self.r3)) #-1 helps us figure out the batchsize
        x1 = self.fc2(F.relu(self.fc1(z1)))
        x2 = self.fc2(F.relu(self.fc1(z2)))
        x = torch.stack([x1,x2])
        x = torch.mean(x,dim=0)
        return x

# Type 4: MLP
class MLP(nn.Module):
    
    def __init__(self, p, N):
        self.p = p
        self.N = N
        super(MLP, self).__init__()
        self.fc1 = nn.Linear(in_features = N*p, out_features = N, bias = True)
    def forward(self, x):
        y = self.fc1(x)
        return y
    
# Type 5: RRR 
class RRR(nn.Module):
    
    def __init__(self, p, N, r1):
        self.p = p
        self.N = N
        self.r1 = r1
        super(RRR, self).__init__()
        self.fc1 = nn.Linear(in_features = N*p, out_features = r1, bias = True)
        self.fc2 = nn.Linear(in_features = r1, out_features = N, bias = True)
    def forward(self, x):
        x = self.fc1(x)
        y = self.fc2(x)
        return y

In [5]:
### "Generate Linear Time Series Inputs and Targetted Output
# return large transition Matrix A
def kronecker(A, B):
    return torch.ger(A.view(-1), B.view(-1)).reshape(*(A.size() + B.size())).permute([0, 2, 1, 3]).reshape(A.size(0)*B.size(0),A.size(1)*B.size(1))

### " F-norm of A
class L2LossFun(nn.Module):
    
    def __init__(self):
        super(L2LossFun, self).__init__()
    def forward(self, A_Est, A_True):
        gap = math.sqrt(torch.sum((A_Est - A_True)**2))
        return gap

class LinfLossFun(nn.Module):
    
    def __init__(self):
        super(LinfLossFun, self).__init__()
    def forward(self, A_Est, A_True):
        gap = max(abs(torch.squeeze(A_Est-A_True))).item()
        return gap
    
def rearrangeG(K):
    K = K.permute(1,0)
    i = j = 0
    for j in range(r3):
            for i in range(r2):
                if i == 0 and j == 0:
                    tmp = K[:1,:]
                else:
                    tmp = torch.cat([tmp, K[(r3*i+j):(r3*i+1+j),:]], dim = 0)   
    tmp = tmp.permute(1,0)
    return(tmp)

def Param_Matrix(net):
    
    U2T = torch.squeeze(net.conv1.weight)
    U3T = torch.squeeze(net.conv2.weight)
    G1 = rearrangeG(net.fc1.weight)
    U1 = net.fc2.weight
    A = torch.mm(torch.mm(U1, G1), kronecker(U3T,U2T))
    
    return A

def Param_Dict(net):
    
    U2T = torch.squeeze(net.conv1.weight)
    U3T = torch.squeeze(net.conv2.weight)
    G1 = rearrangeG(net.fc1.weight)
    U1 = net.fc2.weight
    A = torch.mm(torch.mm(U1, G1), kronecker(U3T,U2T))
    
    Param_List = {
        "U2T" : U2T,
        "U3T" : U3T,
        "G1" : G1,
        "U1" : U1,
        "A" : A
    }
    
    return Param_List

    #A = genA_true.genA(r1, r2, r3, p, N) #generate large transition matrix 

In [6]:
### "We can use our method to generate RandomDataset
# For our linear settings burnt in is needed
class RealDataset(Dataset):
    
    def __init__(self, p, N, Smp_size, df):
        self.X = []
        self.y = []
        for i in range(Smp_size):
            if i == 0:
                input_TS = torch.tensor(df[ [19,18,17,16,15,14,13,12,11,10,9,8,7,6,5,4,3,2,1,0] , : ]).permute(1,0).view(1,1,N,p) #change according to p
                self.X.append(torch.squeeze(input_TS.view(1,1,N,p), dim = 0))
                output_TS = torch.squeeze(torch.tensor(df[ p:(p+1) , : ]).permute(1,0).view(1,1,N,1))
                self.y.append(output_TS[:N])  
            else:
                input_TS = torch.cat([self.y[i-1].view(1,1,N,1), input_TS], dim = 3)
                self.X.append(torch.squeeze(input_TS[:,:,:N,:p], dim = 0))
                out_tmp = torch.squeeze(torch.tensor(df[ (i+p):(i+p+1) , : ]).permute(1,0).view(1,1,N,1))
                self.y.append(out_tmp[:N])
                
    def __getitem__(self, idx):
        return self.X[idx], self.y[idx]
    
    def __len__(self):
        return len(self.X)

In [7]:
# input to GPU
#device = torch.device("cuda:2")
#device2 = torch.device('cpu')

In [8]:
# initialize dict with dynamic list for storage
names = {}

names["y_true"] = []

names["predErrorDW"] =  []
names["predErrorDWLinf"] =  []
names["LTRDW_pred"] = []
names["predErrorN"] =  []
names["predErrorNLinf"] =  []
names["LTRN_pred"] = []
names["predErrorL"] =  []
names["predErrorLLinf"] =  []
names["LTRL_pred"] = []
names["predErrorM"] =  []
names["predErrorMLinf"] =  []
names["LTRM_pred"] = []
names["predErrorR"] =  []
names["predErrorRLinf"] =  []
names["LTRR_pred"] = []

In [ ]:
distance = L2LossFun()
distanceLinf = LinfLossFun()

t1_start = perf_counter() 

for k in range(Pred_size):
    
    if k == 0:
        dsT = RealDataset(p=p, N=N, Smp_size=Smp_size, df=df[k:,])
    else:
        dsT = dsF
    
    dsF = RealDataset(p=p, N=N, Smp_size=Smp_size, df=df[(k+1):,])
    X_F,y_F = dsF[Smp_size-1]
    X_F = X_F.view(1,1,N,p)
    names["y_true"].append(y_F)
    ds = DataLoader(dsT, batch_size=Smp_size, shuffle=False)

    netDW = NetDW(r1, r2, r3, p, N)

    criterion = nn.MSELoss()
    optimizerDW = optim.SGD(netDW.parameters(), lr = 0.01, momentum=0.9)

    loss_last = 1000
    loss_new = 0
    
    ### "" netDW
    i = 0
    while abs(loss_last - loss_new) > 0.000001: 
        if i > 0:
            loss_last = loss_new  
        for ix, (_x, _y) in enumerate(ds):
            #=========make inpur differentiable=======================
            _x = Variable(_x).float()
            _y = torch.squeeze(Variable(_y).float())
            #========forward pass=====================================
            yhat = netDW(_x).float()
            loss = criterion(yhat, _y)
            #=======backward pass=====================================
            optimizerDW.zero_grad() 
            loss.backward() 
            optimizerDW.step() 
            loss_new = loss.item()
        i = i + 1
        
    X_F = Variable(X_F).float()
    y_F = torch.tensor(Variable(y_F).float())
    y_predDW = netDW(X_F).float()
    predErrorDW = distance(y_predDW, y_F)
    names["predErrorDW"].append(predErrorDW)
    print("PredError is {}.".format(predErrorDW))
    predErrorDWLinf = distanceLinf(y_predDW, y_F)
    names["predErrorDWLinf"].append(predErrorDWLinf)
    print("PredErrorLinf is {}.".format(predErrorDWLinf))
    names["LTRDW_pred"].append(y_predDW)
    
    ### "" netReLU
    netN = NetRelu(r1, r2, r3, p, N)

    criterion = nn.MSELoss()
    optimizerN = optim.SGD(netN.parameters(), lr = 0.01, momentum=0.9)

    loss_last = 1000
    loss_new = 0
    i = 0
    while abs(loss_last - loss_new) > 0.000001:
        if i > 0:
            loss_last = loss_new  
        for ix, (_x, _y) in enumerate(ds):
            #=========make inpur differentiable=======================
            _x = Variable(_x).float()
            _y = torch.squeeze(Variable(_y).float())
            #========forward pass=====================================
            yhat = netN(_x).float()
            loss = criterion(yhat, _y)
            #=======backward pass=====================================
            optimizerN.zero_grad() 
            loss.backward() 
            optimizerN.step() 
            loss_new = loss.item()
        i = i + 1
        
    X_F = Variable(X_F).float()
    y_F = torch.tensor(Variable(y_F).float())
    y_predN = netN(X_F).float()
    predErrorN = distance(y_predN, y_F)
    names["predErrorN"].append(predErrorN)
    print("PredError Nonlinear is {}.".format(predErrorN))
    predErrorNLinf = distanceLinf(y_predN, y_F)
    names["predErrorNLinf"].append(predErrorNLinf)
    print("PredErrorLinf Nonlinear is {}.".format(predErrorNLinf))
    names["LTRN_pred"].append(y_predN)
    
    ### "" netLinear
    netL = NetLinear(r1, r2, r3, p, N)

    criterion = nn.MSELoss()
    optimizerL = optim.SGD(netL.parameters(), lr = 0.01, momentum=0.9)

    loss_last = 1000
    loss_new = 0
    i = 0
    while abs(loss_last - loss_new) > 0.000001:
        if i > 0:
            loss_last = loss_new  
        for ix, (_x, _y) in enumerate(ds):
            #=========make inpur differentiable=======================
            _x = Variable(_x).float()
            _y = torch.squeeze(Variable(_y).float())
            #========forward pass=====================================
            yhat = netL(_x).float()
            loss = criterion(yhat, _y)
            #=======backward pass=====================================
            optimizerL.zero_grad() 
            loss.backward() 
            optimizerL.step() 
            loss_new = loss.item()
        i = i + 1
        
    X_F = Variable(X_F).float()
    y_F = torch.tensor(Variable(y_F).float())
    y_predL = netL(X_F).float()
    predErrorL = distance(y_predL, y_F)
    names["predErrorL"].append(predErrorL)
    print("PredError Linear is {}.".format(predErrorL))
    predErrorLLinf = distanceLinf(y_predL, y_F)
    names["predErrorLLinf"].append(predErrorLLinf)
    print("PredErrorLinf Linear is {}.".format(predErrorLLinf))
    names["LTRL_pred"].append(y_predL)
    
    ### "" MLP
    netM = MLP(p, N)

    criterion = nn.MSELoss()
    optimizerM = optim.SGD(netM.parameters(), lr = 0.01, momentum=0.9)

    loss_last = 1000
    loss_new = 0
    i = 0
    while abs(loss_last - loss_new) > 0.000001:
        if i > 0:
            loss_last = loss_new  
        for ix, (_x, _y) in enumerate(ds):
            #=========make inpur differentiable=======================
            _x = torch.squeeze(_x.permute(0,1,3,2).reshape(Smp_size, 1, p*N).permute(1, 0, 2))
            _x = Variable(_x).float()
            _y = torch.squeeze(Variable(_y).float())
            #========forward pass=====================================
            yhat = netM(_x).float()
            loss = criterion(yhat, _y)
            #=======backward pass=====================================
            optimizerM.zero_grad() 
            loss.backward() 
            optimizerM.step() 
            loss_new = loss.item()
        i = i + 1
        
    X_F = Variable(X_F.permute(0,1,3,2).reshape(1, p*N)).float()
    y_F = torch.tensor(Variable(y_F).float())
    y_predM = netM(X_F).float()
    predErrorM = distance(y_predM, y_F)
    names["predErrorM"].append(predErrorM)
    print("PredError MLP is {}.".format(predErrorM))
    predErrorMLinf = distanceLinf(y_predM, y_F)
    names["predErrorMLinf"].append(predErrorMLinf)
    print("PredErrorLinf MLP is {}.".format(predErrorMLinf))
    names["LTRM_pred"].append(y_predM)

    ### "" RRR
    netR = RRR(p, N, r1)

    criterion = nn.MSELoss()
    optimizerR = optim.SGD(netR.parameters(), lr = 0.01, momentum=0.9)

    loss_last = 1000
    loss_new = 0
    i = 0
    while abs(loss_last - loss_new) > 0.000001:
        if i > 0:
            loss_last = loss_new  
        for ix, (_x, _y) in enumerate(ds):
            #=========make inpur differentiable=======================
            _x = torch.squeeze(_x.permute(0,1,3,2).reshape(Smp_size, 1, p*N).permute(1, 0, 2))
            _x = Variable(_x).float()
            _y = torch.squeeze(Variable(_y).float())
            #========forward pass=====================================
            yhat = netR(_x).float()
            loss = criterion(yhat, _y)
            #=======backward pass=====================================
            optimizerR.zero_grad() 
            loss.backward() 
            optimizerR.step() 
            loss_new = loss.item()
        i = i + 1
        
    X_F = Variable(X_F).float()
    y_F = torch.tensor(Variable(y_F).float())
    y_predR = netR(X_F).float()
    predErrorR = distance(y_predR, y_F)
    names["predErrorR"].append(predErrorR)
    print("PredError RRR is {}.".format(predErrorR))
    predErrorRLinf = distanceLinf(y_predR, y_F)
    names["predErrorRLinf"].append(predErrorRLinf)
    print("PredErrorLinf RRR is {}.".format(predErrorRLinf))
    names["LTRR_pred"].append(y_predR)

    print(k)

t1_stop = perf_counter() 
print("Elapsed time during the whole program in seconds:", 
                                        t1_stop-t1_start) 
Real_100_245_20_2s = names
torch.save(Real_100_245_20_2s, "Real_100_245_20_2s.py")

/Users/feiqinghuang/.virtualenvs/dl4cv/lib/python3.6/site-packages/ipykernel_launcher.py:47: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).


PredError is 0.9595042759778677.
PredErrorLinf is 0.4174957871437073.


/Users/feiqinghuang/.virtualenvs/dl4cv/lib/python3.6/site-packages/ipykernel_launcher.py:84: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).


PredError Nonlinear is 0.9842291527134913.
PredErrorLinf Nonlinear is 0.43367964029312134.


/Users/feiqinghuang/.virtualenvs/dl4cv/lib/python3.6/site-packages/ipykernel_launcher.py:121: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).


PredError Linear is 0.9567897569795436.
PredErrorLinf Linear is 0.39854711294174194.


/Users/feiqinghuang/.virtualenvs/dl4cv/lib/python3.6/site-packages/ipykernel_launcher.py:159: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).


PredError MLP is 1.1841891965841351.
PredErrorLinf MLP is 0.4802646040916443.


/Users/feiqinghuang/.virtualenvs/dl4cv/lib/python3.6/site-packages/ipykernel_launcher.py:197: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).


PredError RRR is 1.0168574558050556.
PredErrorLinf RRR is 0.4326026439666748.
0
PredError is 1.200050041029555.
PredErrorLinf is 0.5055556297302246.
PredError Nonlinear is 1.2160351176649502.
PredErrorLinf Nonlinear is 0.5289795994758606.
PredError Linear is 1.3487390563351165.
PredErrorLinf Linear is 0.5368069410324097.


In [232]:
from statistics import mean,median

print(mean(names["predErrorDW"]))
print(mean(names["predErrorDWLinf"]))
print(mean(names["predErrorN"]))
print(mean(names["predErrorNLinf"]))
print(mean(names["predErrorL"]))
print(mean(names["predErrorLLinf"]))
print(mean(names["predErrorM"]))
print(mean(names["predErrorMLinf"]))
print(mean(names["predErrorR"]))
print(mean(names["predErrorRLinf"]))

0.7115289762922289
0.3261041794504438
0.7117751168962091
0.326625252195767
0.807227811429077
0.326625252195767
0.8024731564090585
0.3356005276952471
0.7673578566489941
0.340144198565256
